In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [3]:
!wget $data -O data-week-3.csv 

--2024-03-27 13:12:30--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.01s   

2024-03-27 13:12:31 (74.7 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [4]:
df = pd.read_csv('data-week-3.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')


In [6]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.5,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,yes


In [7]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [8]:
df.shape

(7043, 21)

In [9]:
df[df['totalcharges'].str.contains('_')]['totalcharges'].count()

11

In [10]:
tc = pd.to_numeric(df['totalcharges'] , errors = 'coerce')

In [11]:
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors = 'coerce')


In [12]:
df['totalcharges'] = df['totalcharges'].fillna(0)


In [13]:
df['churn']

0        no
1        no
2       yes
3        no
4       yes
       ... 
7038     no
7039     no
7040     no
7041    yes
7042     no
Name: churn, Length: 7043, dtype: object

In [14]:
(df['churn'] == 'yes').astype(int).head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

In [15]:
df['churn'] = (df['churn'] == 'yes').astype(int)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   seniorcitizen     7043 non-null   int64  
 3   partner           7043 non-null   object 
 4   dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   phoneservice      7043 non-null   object 
 7   multiplelines     7043 non-null   object 
 8   internetservice   7043 non-null   object 
 9   onlinesecurity    7043 non-null   object 
 10  onlinebackup      7043 non-null   object 
 11  deviceprotection  7043 non-null   object 
 12  techsupport       7043 non-null   object 
 13  streamingtv       7043 non-null   object 
 14  streamingmovies   7043 non-null   object 
 15  contract          7043 non-null   object 
 16  paperlessbilling  7043 non-null   object 


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [19]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)


In [20]:
len(df_full_train), len(df_test)

(5634, 1409)

In [21]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)


In [22]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [23]:
df_train.reset_index(drop = True)
df_val.reset_index(drop = True)
df_test.reset_index(drop = True)    

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,1024-guald,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,24.80,24.80,1
1,0484-jpbru,male,0,no,no,41,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,month-to-month,yes,bank_transfer_(automatic),25.25,996.45,0
2,3620-ehimz,female,0,yes,yes,52,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.35,1031.70,0
3,6910-hadcm,female,0,no,no,1,yes,no,fiber_optic,no,...,yes,no,no,no,month-to-month,no,electronic_check,76.35,76.35,1
4,8587-xyzsf,male,0,no,no,67,yes,no,dsl,no,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),50.55,3260.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,2845-kdhvx,female,0,yes,no,64,yes,no,dsl,no,...,yes,yes,no,yes,two_year,yes,mailed_check,68.30,4378.80,0
1405,5027-yocxn,male,0,yes,yes,51,yes,yes,fiber_optic,yes,...,no,yes,yes,yes,one_year,no,credit_card_(automatic),110.05,5686.40,0
1406,8884-mrnsu,male,0,yes,yes,17,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,one_year,no,bank_transfer_(automatic),19.90,329.75,0
1407,0567-xrhcu,female,0,yes,yes,69,no,no_phone_service,dsl,yes,...,yes,no,no,yes,two_year,yes,credit_card_(automatic),43.95,2960.10,0


In [24]:
y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test['churn'].values


In [25]:
df_train.drop(columns = 'churn', inplace = True)
df_val.drop(columns = 'churn', inplace = True)
df_test.drop(columns = 'churn', inplace = True)

In [26]:
df_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
6157,6776-tlwoi,male,0,no,no,3,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,month-to-month,no,mailed_check,19.85,64.55
817,0036-ihmot,female,0,yes,yes,55,yes,no,fiber_optic,no,yes,yes,yes,yes,yes,one_year,yes,bank_transfer_(automatic),103.70,5656.75
6977,2843-cqmeg,male,0,no,no,24,yes,no,dsl,no,yes,no,no,no,no,month-to-month,yes,mailed_check,49.70,1218.25
1932,3247-mhjkm,male,0,no,no,1,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,month-to-month,no,mailed_check,20.20,20.20
4168,1194-bhjyc,male,0,yes,no,61,no,no_phone_service,dsl,yes,no,yes,yes,yes,yes,two_year,yes,mailed_check,62.15,3778.85


# EDA

In [27]:
df_full_train = df_full_train.reset_index(drop = True)


In [28]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [29]:
df_full_train['churn'].value_counts()

churn
0    4138
1    1496
Name: count, dtype: int64

In [30]:
df_full_train['churn'].value_counts(normalize = True)

churn
0    0.734469
1    0.265531
Name: proportion, dtype: float64

In [31]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [32]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [33]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [37]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
      'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod',]

In [38]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

# Feature importance : Churn rate and risk ratio

In [39]:
df_full_train[df_full_train['gender'] == 'female']['churn'].mean()

0.2708409173643975

In [41]:
df_full_train[df_full_train['gender'] == 'male']['churn'].mean()

0.26047800484932454

In [43]:
df_full_train[df_full_train['partner'] == 'yes']['churn'].mean()

0.20073260073260074

In [44]:
df_full_train[df_full_train['partner'] == 'no']['churn'].mean()

0.32644628099173556

In [45]:
df_full_train['churn'].mean()

0.2655307064252751

In [48]:
df_group = df_full_train.groupby('gender')['churn'].agg(['mean', 'count'])

In [49]:
df_group['diff'] = df_group['mean'] - df_full_train['churn'].mean()
df_group['risk'] = df_group['mean'] / df_full_train['churn'].mean()
df_group

,mean,count,diff,risk
gender,,,,
female,0.270841,2747,0.005310,1.019998
male,0.260478,2887,-0.005053,0.980971


In [50]:
global_churn = df_full_train['churn'].mean()

In [51]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.270841,2747,0.005310,1.019998
male,0.260478,2887,-0.005053,0.980971



seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.237098,4728,-0.028433,0.892922
1,0.413907,906,0.148377,1.558793



partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.426533,3083,0.161002,1.606340
one_year,0.117987,1212,-0.147544,0.444343
two_year,0.028379,1339,-0.237151,0.106878



paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.16414,2309,-0.101390,0.618159
yes,0.33594,3325,0.070409,1.265164



paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.174475,1238,-0.091056,0.657080
credit_card_(automatic),0.152404,1227,-0.113126,0.573961
electronic_check,0.449921,1887,0.184390,1.694420
mailed_check,0.190328,1282,-0.075203,0.716782


# Feature importance: Mutual information

In [52]:
from sklearn.metrics import mutual_info_score

In [53]:
mutual_info_score(df_full_train['churn'], df_full_train['contract'])

0.0966515250997

In [54]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train['churn'])

In [55]:
df_full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending = False)

contract            0.096652
onlinesecurity      0.063393
techsupport         0.060935
internetservice     0.053313
onlinebackup        0.045424
paymentmethod       0.042861
deviceprotection    0.042007
streamingtv         0.030844
streamingmovies     0.030705
paperlessbilling    0.019077
dependents          0.014129
partner             0.010227
seniorcitizen       0.010059
multiplelines       0.000654
gender              0.000069
phoneservice        0.000040
dtype: float64

# Feature importance: Correlation

In [56]:
df_full_train[numerical].corrwith(df_full_train['churn'])

tenure           -0.344925
monthlycharges    0.188574
totalcharges     -0.193370
dtype: float64

In [58]:
df_full_train[df_full_train['tenure'] > 2]['churn'].mean()

0.22295938882187374

# One-hot encoding

In [59]:
from sklearn.feature_extraction import DictVectorizer

In [75]:
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')

In [76]:
train_dicts[0]

{'gender': 'male',
 'seniorcitizen': 0,
 'partner': 'no',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'no',
 'onlinesecurity': 'no_internet_service',
 'onlinebackup': 'no_internet_service',
 'deviceprotection': 'no_internet_service',
 'techsupport': 'no_internet_service',
 'streamingtv': 'no_internet_service',
 'streamingmovies': 'no_internet_service',
 'contract': 'month-to-month',
 'paperlessbilling': 'no',
 'paymentmethod': 'mailed_check',
 'tenure': 3,
 'monthlycharges': 19.85,
 'totalcharges': 64.55}

In [77]:
dv = DictVectorizer(sparse= False)

In [79]:
dv.fit_transform(train_dicts)

array([[1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        3.00000e+00, 6.45500e+01],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        5.50000e+01, 5.65675e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        2.40000e+01, 1.21825e+03],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        1.50000e+01, 6.03000e+02],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        4.50000e+01, 9.03700e+02],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        2.90000e+01, 1.80110e+03]])

In [83]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

In [84]:
X_train = dv.fit_transform(train_dicts)

In [85]:
X_train.shape

(4225, 45)

In [86]:
val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')

In [87]:
X_val = dv.transform(val_dicts)

In [88]:
test_dicts = df_test[categorical + numerical].to_dict(orient = 'records')


In [89]:
X_test = dv.transform(test_dicts)

# Training logistice regression with Scikit-learn

In [90]:
from sklearn.linear_model import LogisticRegression

In [91]:
model = LogisticRegression()
model.fit(X_train, y_train)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [92]:
model.coef_[0].round(3)

array([ 0.549, -0.206, -0.469, -0.052, -0.074,  0.035, -0.114, -0.047,
        0.027, -0.154, -0.305,  0.292, -0.114,  0.002, -0.214,  0.123,
       -0.036,  0.088, -0.114, -0.1  ,  0.237, -0.114, -0.25 , -0.234,
        0.107, -0.104, -0.023, -0.112, -0.181,  0.254, -0.088,  0.123,
       -0.25 ,  0.176, -0.15 , -0.114,  0.138, -0.066, -0.114,  0.053,
        0.201, -0.114, -0.214, -0.059,  0.   ])

In [97]:
(model.predict(X_train) == 0).sum()

3295

In [99]:
y_pred = model.predict_proba(X_val)[:, 1]

In [101]:
churn_decision = (y_pred >= 0.5)

In [103]:
churn_decision

array([False, False, False, ...,  True, False, False])

In [102]:
df_val[churn_decision]

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
3334,0337-cnpze,female,0,no,no,41,yes,yes,fiber_optic,no,yes,yes,no,yes,yes,month-to-month,yes,electronic_check,106.30,4443.45
2173,2038-oeqzh,male,0,no,no,4,yes,no,fiber_optic,no,no,yes,no,no,yes,month-to-month,no,electronic_check,86.05,308.10
5831,9846-gkxas,female,0,no,no,9,yes,no,fiber_optic,no,no,no,no,yes,yes,month-to-month,yes,bank_transfer_(automatic),90.80,809.75
2133,8051-hjrlt,female,1,no,no,1,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.55,70.55
5479,8990-zxlsu,female,0,no,no,9,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,no,electronic_check,69.05,651.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6440,2809-zmyoq,female,0,no,no,32,yes,yes,fiber_optic,no,no,yes,no,yes,yes,month-to-month,yes,electronic_check,101.35,3334.90
1821,3536-iqctx,male,1,yes,no,32,yes,yes,fiber_optic,no,no,no,no,no,yes,month-to-month,no,electronic_check,85.30,2661.10
5706,5196-sgoak,female,0,no,no,1,yes,no,fiber_optic,no,yes,no,no,no,no,month-to-month,yes,electronic_check,75.70,75.70
875,3050-gbush,female,0,no,no,3,no,no_phone_service,dsl,no,no,no,no,yes,no,month-to-month,yes,credit_card_(automatic),34.80,113.95


In [104]:
(y_val == churn_decision).mean()

0.801277501774308

In [105]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = churn_decision.astype(int)
df_pred['actual'] = y_val

In [106]:
df_pred.head()

,probability,prediction,actual
0,0.148190,0,0
1,0.245720,0,0
2,0.410734,0,1
3,0.559695,1,1
4,0.049875,0,0


In [107]:
df_pred['correct'] = df_pred['prediction'] == df_pred['actual']

In [108]:
df_pred

,probability,prediction,actual,correct
0,0.148190,0,0,True
1,0.245720,0,0,True
2,0.410734,0,1,False
3,0.559695,1,1,True
4,0.049875,0,0,True
...,...,...,...,...
1404,0.066813,0,1,False
1405,0.311372,0,0,True
1406,0.717757,1,1,True
1407,0.058537,0,0,True


In [109]:
df_pred['correct'].mean()

0.801277501774308

# Model implementation

In [110]:
a = [1, 2, 3, 4]
b=  'abcd'

In [111]:
list(zip(a, b))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

In [112]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.549,
 'contract=one_year': -0.206,
 'contract=two_year': -0.469,
 'dependents=no': -0.052,
 'dependents=yes': -0.074,
 'deviceprotection=no': 0.035,
 'deviceprotection=no_internet_service': -0.114,
 'deviceprotection=yes': -0.047,
 'gender=female': 0.027,
 'gender=male': -0.154,
 'internetservice=dsl': -0.305,
 'internetservice=fiber_optic': 0.292,
 'internetservice=no': -0.114,
 'monthlycharges': 0.002,
 'multiplelines=no': -0.214,
 'multiplelines=no_phone_service': 0.123,
 'multiplelines=yes': -0.036,
 'onlinebackup=no': 0.088,
 'onlinebackup=no_internet_service': -0.114,
 'onlinebackup=yes': -0.1,
 'onlinesecurity=no': 0.237,
 'onlinesecurity=no_internet_service': -0.114,
 'onlinesecurity=yes': -0.25,
 'paperlessbilling=no': -0.234,
 'paperlessbilling=yes': 0.107,
 'partner=no': -0.104,
 'partner=yes': -0.023,
 'paymentmethod=bank_transfer_(automatic)': -0.112,
 'paymentmethod=credit_card_(automatic)': -0.181,
 'paymentmethod=electronic_check': 0.254,
 

In [113]:
small = ['contract', 'tenure', 'monthlycharges']


In [115]:
df_train[small].iloc[:10].to_dict(orient = 'records')

[{'contract': 'month-to-month', 'tenure': 3, 'monthlycharges': 19.85},
 {'contract': 'one_year', 'tenure': 55, 'monthlycharges': 103.7},
 {'contract': 'month-to-month', 'tenure': 24, 'monthlycharges': 49.7},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 20.2},
 {'contract': 'two_year', 'tenure': 61, 'monthlycharges': 62.15},
 {'contract': 'two_year', 'tenure': 42, 'monthlycharges': 20.7},
 {'contract': 'month-to-month', 'tenure': 23, 'monthlycharges': 54.15},
 {'contract': 'month-to-month', 'tenure': 37, 'monthlycharges': 98.8},
 {'contract': 'one_year', 'tenure': 43, 'monthlycharges': 20.0},
 {'contract': 'month-to-month', 'tenure': 20, 'monthlycharges': 73.65}]

In [116]:
dicts_train_small = df_train[small].to_dict(orient = 'records')
dicts_val_small = df_val[small].to_dict(orient = 'records')
dicts_test_small = df_test[small].to_dict(orient = 'records')

In [117]:
dv_small = DictVectorizer(sparse = False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [118]:
dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [119]:
X_train_small = dv_small.transform(dicts_train_small)

In [120]:
model_small = LogisticRegression()
model_small.fit(X_train_small, y_train)

LogisticRegression()

In [123]:
w0 =model_small.intercept_[0]
w0

-2.565639898493251

In [125]:
w = model_small.coef_[0]
w.round(3)

array([ 1.067, -0.026, -1.032,  0.027, -0.036])

In [126]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))

{'contract=month-to-month': 1.067,
 'contract=one_year': -0.026,
 'contract=two_year': -1.032,
 'monthlycharges': 0.027,
 'tenure': -0.036}

# Using the model